### University of Toronto, Department of Electrical and Computer Engineering

## ECE 1501 &mdash; Error Control Codes

# Exercise 1:  Introduction to Binary Linear Codes

### Last name: 
### First name: 
### Student number:

Welcome to the numerical exercise of Module 1 in ECE1501!

The purpose of this first Numerical Exercise is to gain experience with binary linear codes.  There are four parts:

1. **Generator Matrices and Parity-Check Matrices:**  You will write a function that takes one to the other.

2. **Weight Enumerators:**  You will compute the weight enumerator for various codes.  You will see that the weight enumerator of a random linear code tends to look like a binomial distribution.  You will also see that random codes usually have poor minimum Hamming distance compared with some good constructions.

3. **Hamming Codes:**  You will generalize the (7,4) binary Hamming code, and build decoders for error-correction and for erasure correction.

4. **Hamming Product Codes:**  You will create a product with Hamming constituent codes, and simulate its performance on the binary symmetric channel.

Each exercise is prefaced by some introductory remarks to help you complete that exercise.

---

# Part 0

## Preliminaries

Run the following cell to load the `Galois2` module by Reza Rafie Borujeny.  See the accompanying [website](https://www.comm.utoronto.ca/~rrafie/ECE1501/Galois2.html) for documentation!

In [ ]:
include("Galois2.jl"); using .Galois2

We will also using the following packages later on.  In case these packages have not yet been installed on your system, uncomment the relevant lines and use `Pkg.add` to add them.

In [ ]:
# import Pkg
# Pkg.add("Plots")
# Pkg.add("Images")
# Pkg.add("FileIO")
# Pkg.add("ImageMagick")
#
using Plots, Images, FileIO, ImageMagick # this may take a little while.  Meanwhile, you may start reading Part 1.

---

# Part 1

## Converting between Generator and Parity-Check Matrices ##

Some useful facts:

1. If $G_{sys}= [I_k \mid P] $ is a $k \times n$ generator matrix in systematic form, then $H_{sys} = [ -P^{\intercal} \mid I_{n-k} ]$ is a corresponding parity-check matrix.

2. An $n \times n$ permutation matrix $Q$ is a matrix with entries from $\{ 0, 1 \}$ having exactly one nonzero entry in each row and column.  If $Q$ is a permutation matrix, then $Q Q^{\intercal} = Q^{\intercal} Q = I_n$.  If $G$ is a $k \times n$ matrix, then the product $GQ$ has the same columns as $G$, except in a permuted order, i.e., multiplication on the right by $Q$ performs a column permutation.  For example, $$ \left[ \begin{array}{ccc}a & b & c \\ d & e & f \end{array} \right] \left[ \begin{array}{ccc} 1 & 0 & 0 \\ 0 & 0 & 1 \\ 0 & 1 & 0 \end{array} \right] = \left[ \begin{array}{ccc} a & c & b \\ d & f & e \end{array} \right] .$$  Note that permutation matrices are well defined over any field.

3. Let $F$ be any field.  Suppose $G \in F^{k \times n}$ and  $H \in F^{(n-k) \times n}$ are matrices, and $Q \in F^{n \times n}$ is a permutation matrix.   Then $$GQ (HQ)^\intercal = G Q Q^{\intercal} H^{\intercal} = GH^{\intercal}.$$   Thus, in particular, if $G H^{\intercal} = 0$, then $(GQ)(HQ)^{\intercal} = 0$.

4. Now let $G$ be a matrix in reduced row echelon form.  For some permutation matrix $Q$, we can write $G_{sys} = GQ$ (i.e., by permuting columns, we can convert from rref to systematic form).  Let $H_{sys}$ satisfy
$G_{sys}H_{sys}^{\intercal} = 0$.  By writing $H_{sys} = HQ$, we also have $GH^{\intercal}=0$.  Thus we can recover $H$ from $H_{sys}$ by computing $$H = H_{sys} Q^{\intercal}.$$  In other words, in principle we can compute $H$ from $G$ by first reducing $G$ to rref, permuting columns (if needed) to convert $G$ to systematic form $G_{sys}$, computing $H_{sys}$, and then performing the inverse permutation on columns to recover $H$ from $H_{sys}$.

5. To avoid a lot of unnecessary data movement caused by permuting columns, observe that in going from $G_{sys}$ to $H_{sys}$ the columns corresponding to $I_k$ are replaced with the columns of $-P^{\intercal}$, while the columns corresponding to $P$ are replaced with the columns of $I_{n-k}$.  Thus, rather than moving columns around, we can simply work in-place, provided that we know which columns of $G$ correspond to $I_k$ and which columns of $G$ correspond to $P$.  For example, suppose $G$ has the following rref:  $$ G = \left[ \begin{array}{ccccc} 1 & a & 0 & 0 & b \\ 0 & 0 & 1 & 0 & c \\ \underbrace{0}_u & \underbrace{0}_p & \underbrace{0}_u & \underbrace{1}_u & \underbrace{d}_p \end{array} \right].$$  The columns corresponding to $I_k$ are labelled with $u$ and the columns corresponding to $P$ are labelled with $p$.  We can then compute a parity-check matrix $H$ directly as $$ H = \left[ \begin{array}{ccccc} -a & 1 & 0 & 0 & 0 \\ -b & 0 & -c & -d & 1 \end{array}\right],$$ where the columns of $-P^{\intercal}$ are placed (in order) in the $u$ columns and the columns of $I_{n-k}$ are placed (in order) in the $p$ columns.


Programming notes:

1. The function ```rref!``` provided by ```Galois2``` mutates a matrix to reduced row echelon form.  The function ```rref``` (without the ```!```) returns the reduced row echelon form of a matrix without mutating its argument.
2. Julia's ```view``` capability can be used to read from and write to selected submatrices of a matrix. Run the following cells and observe the output.
3. Julia's ```findfirst``` function can be used to find the index of the pivot element in the ```i```th row of a matrix ```G``` as follows: ```findfirst(x->x!=GF2(0),G[i,:])```;  this expression produces ```nothing``` in case the row is all-zero.

In [ ]:
G = GF2[1 1 1 1 0; 0 0 1 1 1; 1 1 0 1 1]  # create a binary matrix

In [ ]:
A = rref(G) # make a copy and reduce to reduced row echelon form

In [ ]:
B = view(A,:,[2,5]) # B is a "view" into columns 2 and 5 of A

In [ ]:
B[3,2] += GF2(1)  # change an element of B...

In [ ]:
A  # ... and note that the corresponding element of A has changed.

In [ ]:
B[3,2] += GF2(1) # undo the previous change

In [ ]:
C = zeros(GF2,2,5)  # create another matrix, filled with zeros

In [ ]:
D = view(C,:,[1,3,4])  # D is a "view" into columns 1, 3, and 4 of C

In [ ]:
copyto!(D,-transpose(B))  # copy the transpose of B into D ... ;  the minus sign is irrelevant in GF(2)

In [ ]:
C # ... and note the effect on C

In [ ]:
C[1,2] = C[2,5] = GF2(1); C  # put unit vectors into the other columns of C

In [ ]:
G * transpose(C)  #  C is a parity-check matrix for G!  Cool!

## **<font color=green>Exercise 1:</font>**

---

1. Write a function ```dual(G::Array{GF2,2})``` that takes in a generator matrix ```G``` for a binary linear $(n,k)$ code and produces a corresponding parity-check matrix.  Your function should handle the case when the input matrix is not of full rank.  If the input matrix has rank n, you should return a matrix of 0 rows and n columns.
2. Test  your function by running the cells below.

In [ ]:
function dual(G::Array{GF2,2})
## to be filled in...
end

In [ ]:
G = GF2[1 1 1 1 0; 0 0 1 1 1; 1 1 0 1 1]  # create a binary matrix (same example as earlier)

In [ ]:
H = dual(G)

In [ ]:
G = ones(GF2,2,5)  # a non-full-rank example

In [ ]:
H = dual(G)

In [ ]:
dual(dual(GF2[0 0 0]))  # another corner case;  should give 0x3 Array{Gf2_1,2}

In [ ]:
G * transpose(H)

In [ ]:
H = GF2[0 0 0 1 1 1 1; 0 1 1 0 0 1 1; 1 0 1 0 1 0 1]  # parity-check matrix for the (7,4) Hamming code

In [ ]:
G = dual(H)  # compute a generator matrix

In [ ]:
rref!(G)  # convert to reduced row echelon form

In [ ]:
G * transpose(H) == zeros(GF2,4,3)  # expect true

In [ ]:
rref(dual(dual(G))) == rref(G)  # expect true

In [ ]:
G = GF2[ 1 1 1 1 1 1 1 1 ; 1 1 1 1 0 0 0 0; 1 1 0 0 1 1 0 0; 1 0 1 0 1 0 1 0]

Let $C$ be the code generated by the matrix ```G``` defined in the previous cell.  Is $C$ self-dual?  (Create cells below this, as needed, to verify your answer.)

---

# Part 2

## Weight enumerator

The weight enumerator for a linear code $C$ of length $n$ is the polynomial $$A_C(x) = \sum_{v \in C} x^{wt(v)} = \sum_{w= 0}^n A_wx^w$$ where $A_w$ is the number of codewords of Hamming weight $w$ in $C$. In software, such a polynomial can be represented by a vector of length $n+1$ $$A_C = (A_0, A_1, A_2, \dots, A_n).$$ In this exercise you will implement a function to find the weight enumerator of a code given its generator matrix. The minimum Hamming distance of a code is the smallest positive index $d$ in its weight enumerator such that $A_d \neq 0$.

## **<font color=green>Exercise 2:</font>**

---

1. Complete the following cell to implement a function that takes in a generator matrix of a binary linear code and produces the weight enumerator of the corresponding code. The returned weight enumerator should be vector so that the element indexed by `i` is the number of codewords of Hamming weight `i-1` in the code.  **Hint:** You may find the function `messagevec` from Numerical Exercise 0 useful!


In [ ]:
function weight_enumerator(G::Array{GF2,2})
    (k, n) = size(G)
    v = zeros(Int, n + 1)
    #=
     complete the function to obtain the weight enumerator of the code generated by G
    =#
    return v
end

2. Run the following code to find the weight enumerator of the (7,4) binary Hamming code:

In [ ]:
G_7_4 = GF2[ 1 0 0 0 0 1 1; 0 1 0 0 1 0 1; 0 0 1 0 1 1 0; 0 0 0 1 1 1 1]
v = weight_enumerator(G_7_4)

3. Find and plot the weight distribution <u>__AND__</u> find the minimum Hamming distance of the linear code generated by the following generator matrix (this code is known as the (24,12) extended binary Golay code. See [here](https://en.wikipedia.org/wiki/Binary_Golay_code) for more about this code!).  An example plot is given below.

In [ ]:
G = GF2[1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1;
        0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0;
        0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 1;
        0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 1 0;
        0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 1 0 0 1;
        0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 1 0 1;
        0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 1 1;
        0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 0;
        0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0;
        0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 1 0;
        0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 0 1;
        0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1];

In [ ]:
v = weight_enumerator(G);
plot(0:length(v)-1, v, line=:stem, marker=:circle, markersize=4, xlabel="\$w\$", ylabel="\$A_w\$", label="Weight Distribution")

In [ ]:
dmin = ## fill in the relevant expression;
println("The minimum Hamming distance is $(dmin)")

4. Generate a random generator matrix for an (24, 12) linear code, plot its weight distribution, and find its minimum distance.

In [ ]:
k = 12
n = 24
G = rand(GF2[0, 1], (k, n))
v = weight_enumerator(G)
plot(0:n, v, line=:stem, marker=:circle, markersize=4, xlabel="\$w\$", ylabel="\$A_w\$", label="")

In [ ]:
dmin = findfirst(x->x != 0,v[2:length(v)])

5. Repeat the above exercise 1000 times; plot the average of the weight enumerators you obtain <u>__AS WELL AS__</u> the relative frequency of getting a code with minimum Hamming distance $d_{\text{min}}$ in your 1000 trials.

6. Compare the best minimum distance in your trials in part 4 with the minimum distance of the (24, 12) extended binary Golay code. 

### The maximum minimum-distance of my random codes is ____  ?? ____, which is <u>smaller</u>/larger than that of the (24, 12) extended Golay code.

---

# Part 3

## Binary Hamming Codes
Binary Hamming codes are characterized by a parameter $m$. For each integer $m\geq 2$, the binary Hamming code with parameter $m$ is a linear code with length $n = 2^m − 1$ and dimension $k = 2^m − m − 1$. Hence the rate of the binary Hamming code with parameter $m$ is $$R = \frac{k}{n} = 1 − \frac{m}{2^m − 1},$$ which is the highest possible for binary linear codes with minimum Hamming distance of 3 and length $2^m − 1$.

A parity-check matrix for a Hamming code can be obtained by taking all distinct nonzero vectors in $\mathbb{F}_2^m$ as columns.  The columns can be arranged in any convenient order.

## **<font color=green>Exercise 3:</font>**

---

1. Write functions `G_Hamming(m)`  and `H_Hamming(m)` that take in a positive integer $m$ and return, respectively, a systematic generator matrix $G$ and a corresponding parity-check matrix $H$ for a $(2^m-1, 2^m-m-1)$ binary Hamming code $C$.

*Hint*: Make use of your previously developed ```dual``` function to implement one of these functions in terms of the other.

*Hint*: How can you guarantee that $G$ will have systematic form?  Just make sure that the last $m$ columns of $H$ are linearly independent.  This guarantees that $C^{\perp}$ has a generator matrix of the form $[ -P^{\intercal} \mid I ]$, which in turn guarantees that $C$ has a generator matrix of the form $[ I \mid P]$.

In [ ]:
function H_Hamming(m::Integer)
    # fill in as needed
end

In [ ]:
function G_Hamming(m)
    # fill in as needed
end

In [ ]:
G_Hamming(2)  # also known as the (3,1) repetition code

In [ ]:
H_Hamming(4)

In [ ]:
G_Hamming(4)

In [ ]:
H_Hamming(4) * transpose(G_Hamming(4))

In [ ]:
G_Hamming(4) * transpose(dual(G_Hamming(4)))

2. Implement a decoder for a binary Hamming code by providing a function ```DEC_Hamming(c::Array{GF2,1},H::Array{GF2,2})``` that maps a binary column vector ```c``` to a Hamming codeword ```v``` satisfing ```H*v = 0```, such that the Hamming distance between ```c``` and ```v``` is at most one.  You may assume that ```H``` is the parity-check matrix for a Hamming code and that the length of ```c``` and the number of columns of ```H``` are the same.  (Note that this function assumes that ```c``` is given as a column vector, not a row vector, since Julia prefers columns.)

In [ ]:
function DEC_Hamming(c::Array{GF2,1},H::Array{GF2,2})
    # fill in as needed
end

3. Run the following code to test your decoder.  It should produce zero errors.

In [ ]:
TRIALS = 1000 # change to a smaller number if you're still debugging
counter = 0
for m = 2:5
    n = 2^m - 1
    k = n - m
    u = rand(GF2[0, 1], (1, k))
    Gt = transpose(G_Hamming(m))
    H = H_Hamming(m)
    for trial in 1:TRIALS
       u = rand(GF2[0,1],k)
       c = Gt * u
       y = copy(c)
       y[rand(1:n)] += GF2(1) # add an error of weight 1
       if c != DEC_Hamming(y,H)
            println("ERROR! m = $(m), c = $(c)")
            counter += 1
        end
    end
end
println("Counted $counter errors.")


4. A binary Hamming code has minimum Hamming distance 3 and therefore is capable of correcting any 2 _erasures_. In this exercise, you are asked to provide an erasure decoder for binary Hamming codes. Let `c` be an integer column vector of length $n = 2^m-1$ with entries from the set $\{-1, 0, 1\}$ such that at most two of its entries are $-1$. Each $-1$ represents an erasure. Write a function `Erasure_DEC_Hamming(c::Array{Int64,1},H::Array{GF2,2})` that takes an input column vector $c$ which is equal to a Hamming codeword $v$ in all positions, except possibly for a maximum of 2 erasure positions and returns $v$.

**Hint:** Replace the erasures with zeros.  If you have a valid codeword, you're done.  Otherwise, use your `DEC_Hamming` function.  If one of the zeros changes to a one, you're done.  If not, your decoder must have corrected elsewhere, and the erasures should be replaced with ones.  You're done.

**Hint:** ```findall(x->x<0, c)``` provides a list of ```i``` where ```c[i]``` is negative. ```GF2.(c)``` converts ```c``` to a vector of ```GF2``` elements (provided that ```c``` contains only zeros and ones).

In [ ]:
function Erasure_DEC_Hamming(y::Array{Int64,1},H::Array{GF2,2})
        # fill in as needed
end

5. Run the following cell to test your erasure decoder:

In [ ]:
TRIALS = 1000 # Change to a smaller number if you're still debugging.

counter0 = 0
counter1 = 0
counter2 = 0

for m = 2:5
    n = 2^m-1
    k = n-m
    Gt = transpose(G_Hamming(m))
    H = H_Hamming(m)
    for trial = 1:TRIALS
        u = rand(GF2[0,1],k)
        c = Gt * u  # channel input
        y = zeros(Int64,n)
        for i=1:n
            y[i] = c[i] == GF2(0) ? 0 : 1
        end  # y now contains the channel output with no erasures
        if c != Erasure_DEC_Hamming(y,H)
            println("ERROR! m = $(m), u = $(u), number of erasures = 0")
            counter0 += 1
        end
        y[rand(1:n)] = -1 # y now contains the channel output with one erasure
        if c != Erasure_DEC_Hamming(y,H)
            println("ERROR! m = $(m), u = $(u), number of erasures = 1")
            counter1 += 1
        end
        while true
            pos = rand(1:n)
            if y[pos] != -1
                y[pos] = -1
                break
            end
        end  # y now contains the channel output with two erasures
        if c != Erasure_DEC_Hamming(y,H)
            println("ERROR! m = $(m), u = $(u), number of erasures = 2")
            counter2 += 1
        end
    end
end
println("Counted $(counter0), $(counter1), and $(counter2) miscorrections with 0, 1, and 2 erasures, respectively.")

---

# Part 4

## Product Codes
<img width="500" src="product.png">

In a binary *product code*, each codeword is an $n_1 \times n_2$ matrix, constrained so that each column forms a codeword of a binary $(n_1,k_1)$ code $C_1$ with generator matrix $G_1$ and each row forms a codeword of a binary $(n_2,k_2)$ code $C_2$ with generator matrix $G_2$.  Such a product code is sometimes denoted as $C_1 \otimes C_2$.

An *encoder* for a product code takes in a $k_1 \times k_2$ binary matrix $U$ of information bits and produces the codeword $$ V = G_1^{\intercal} U G_2. $$  When $G_1$ and $G_2$ both are systematic, each codeword $V$ takes the form as shown in the diagram above, where the $k_1 \times k_2$ submatrix labelled "information bits" corresponds to $U$.

There are many possible decoding algorithms for a product code $C_1 \otimes C_2$.  Often, decoders are built from "local" decoders for $C_1$ and $C_2$, which look at a single column or row of the received matrix at a time.  By iterating between row decoders and column decoders, the received matrix can often be transformed into a nearby codeword. 

## **<font color=green>Exercise 4:</font>**

---


1. Write a function `ENC_Product(u::Array{GF2,2},G1::Array{GF2,2},G2::Array{GF2,2})` that maps a binary $k_1 \times k_2$ ```u``` to a product codeword in which every column is a codeword of the code generated by ```G1``` and every column is a codeword of the code generated by ```G2```.

In [ ]:
function ENC_Product(u::Array{GF2,2},G1::Array{GF2,2},G2::Array{GF2,2})
    # fill in as needed
end     

2. Run the following cell to encode the identity matrix using a product of (7,4) Hamming codes.  

In [ ]:
G1 = G2 = G_Hamming(3)
u = zeros(GF2,4,4)
for i = 1:4
    u[i, i] = GF2(1)
end
c = ENC_Product(u,G1,G2)

3. Implement a decoder `DEC_Product_Hamming(c::Array{GF2,2},H1::Array{GF2,2},H2::Array{GF2,2},itr::Integer)` that takes a binary matrix `c` of size $n_1 \times n_2$ as input and produces a binary matrix `u` by doing the following steps:

    i)   Mutate each *column* by flipping at most 1 bit to make it a valid codeword, i.e., satisfying zero syndrome with respect to parity-check matrix `H1`.
    
    ii)  Mutate each *row* by flipping at most 1 bit to make it a valid codeword, i.e., satisfying zero syndrome with respect to parity-check matrix ```H2```.
    
    iii) Repeat until all rows and all columns are valid codewords or declare a decoding failure after `itr` iterations.


In [ ]:
function DEC_Product_Hamming(c::Array{GF2,2},H1::Array{GF2,2},H2::Array{GF2,2},itr::Integer)
    # fill in as needed
end   
    

In [ ]:
G1 = G_Hamming(3); G2 = G_Hamming(3); H1 = H_Hamming(3); H2 = H_Hamming(3)

In [ ]:
u = zeros(GF2,4,4); u[1,1]=u[2,2]=u[3,3]=u[4,4]=GF2(1); v = ENC_Product(u,G1,G2)

In [ ]:
v[3,4] += GF2(1); v[1,1] += GF2(1); v[3,3] += GF2(1); v

In [ ]:
DEC_Product_Hamming(v,H1,H2,3)

4. Test your decoder by running the following cell:

In [ ]:
u = GF2.(zeros(Int, 4, 4))
for i = 1:4
    u[i, i] = GF2(1)
end
G1 = G2 = G_Hamming(3)
H1 = H2 = H_Hamming(3)
c = ENC_Product(u,G1,G2)

itr = 10 # max number of decoding iterations.

counter1 = 0
for i = 1:7
    for j = 1:7
        c_temp = copy(c)
        c_temp[i, j] += GF2(1)  # random single error
        c_decoded = DEC_Product_Hamming(c_temp, H1,H2,itr)
        if c != c_decoded
            counter1 += 1
        end
    end
end

counter2 = 0
trials = 10
for i = 1:trials
    c_temp = copy(c)
    c_temp[rand(1:7*7)] += GF2(1)  # random double error
    c_temp[rand(1:7*7)] += GF2(1)
    c_decoded = DEC_Product_Hamming(c_temp, H1,H2,itr)
    if c != c_decoded
        counter2 += 1
    end
end


counter3 = 0
trials = 10
for i = 1:trials
    c_temp = copy(c)
    c_temp[rand(1:7*7)] += GF2(1) # random triple error
    c_temp[rand(1:7*7)] += GF2(1)
    c_temp[rand(1:7*7)] += GF2(1)
    c_decoded = DEC_Product_Hamming(c_temp, H1,H2,itr)
    if c != c_decoded
        counter3 += 1
    end
end

counter4 = 0
trials = 10
for i = 1:trials
    c_temp = copy(c)
    x1 = rand(1:7)
    y1 = rand(1:7)
    x2 = rand(1:7)
    y2 = rand(1:7)
    while x2 == x1
        x2 = rand(1:7)
    end
    while y2 == y1
        y2 = rand(1:7)
    end
    c_temp[x1, y1] += GF2(1)  # an uncorrectable pattern
    c_temp[x1, y2] += GF2(1)
    c_temp[x2, y1] += GF2(1)
    c_temp[x2, y2] += GF2(1)

    c_decoded = DEC_Product_Hamming(c_temp, H1,H2,itr)
    if c != c_decoded
        counter4 += 1
    end
end

[counter1, counter2, counter3, counter4]


5. Read in the image file `spider_man.png` by running the following cell:

In [ ]:
img = Gray.(load("spider_man.png"))

6. Simulate the transmission of `spider_man.png` over a binary symetric channel with crossover probability `p = 0.002`.

In [ ]:
pic = GF2.(map(x -> Int(x > 0.6),  img))  # convert to bits

p = 0.002
noisy = (map(x -> x + (rand()<p ? GF2(1) : GF2(0)), pic))
Gray.(map(x-> x==GF2(1) ? 1 : 0, noisy))

7. Repeat the above experiment, but this time first encode the picture using `ENC_Product` and, after transmission, decode the received data using `DEC_Product_Hamming` and display the results.  You should display the transmitted codeword, the received word, and the output of the decoder.  You can measure the size of the image via `size(img)` and select the appropriate Hamming code parameters.

In [ ]:
# write code to produce an image showing the transmitted codeword

In [ ]:
# write code to produce an image showing the received word

In [ ]:
# write code to produce an image showing the decoded word

8. What happens when $p$ is larger?  Repeat the experiment for `p=0.003`, `p=0.005`.  You only need to show the decoded images.

---

This completes Numerical Exercise 1.  Following the same directions as in Exercise 0, convert to html, and the print to PDF to create a file that can be uploaded on Quercus on or before the due date.